<a href="https://colab.research.google.com/github/semenko/liquid-cell-atlas/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Necessary Libraries

In [1]:
!pip install pyBigWig pybedtools gunzip
!apt install bedtools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 957 kB/s 
     |████████████████████████████████| 12.5 MB 7.2 MB/s 
     |████████████████████████████████| 15.1 MB 19.6 MB/s 
  Created wheel for pyBigWig: filename=pyBigWig-0.3.18-cp37-cp37m-linux_x86_64.whl size=196937 sha256=9e14c71d32334535148abf4abd4c50a083f8c17dafce8096f09a0bc19175b4f7
  Stored in directory: /root/.cache/pip/wheels/28/eb/46/c761563ba38bd516bcc6accde3d4188cd84eec067f9201cbec
  Created wheel for pybedtools: filename=pybedtools-0.9.0-cp37-cp37m-linux_x86_64.whl size=13616821 sha256=b14c4edb5799009875504c8f15d8b5efdfd1578f49f00ca19d3d0fba3aa67c1a
  Stored in directory: /root/.cache/pip/wheels/7a/44/0d/3a7449885adaf8ebb157da8c3c834a712f48b3b3b84ba51dda
Successfully built pyBigWig pybedtools
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically 

In [2]:
import pandas as pd
from google.colab import files
import io
import json
import itertools
import numpy as np
import altair as alt
from tqdm.notebook import tqdm
import csv
import os
import urllib
import pandas as pd
import altair as alt
from altair import data_transformers
import pickle
from google.colab import drive

import pyBigWig
import pybedtools

drive.mount("/content/drive")

Mounted at /content/drive


# Get TSV of all data from Blueprint, Filter and Download it

In [ ]:
# Download the TSV file from http://dcc.blueprint-epigenome.eu/#/files, and upload it here
# file = files.upload()
! wget 'http://dcc.blueprint-epigenome.eu/data/blueprint_files.tsv'
data_tsv = pd.read_csv('blueprint_files.tsv', sep='\t')

Filtering the tsv file to get rid of individuals with diseases, and only keeping the bigWig file format. We also only keep bisulfite sequencing data.

In [ ]:
noDisease_bw_data = data_tsv[(data_tsv['Disease'] == 'None') & 
                             (data_tsv['Format'] == 'bigWig') & 
                             (data_tsv['Experiment'] == 'Bisulfite-Seq')]

Types of cells present in the dataset. For now, I'll choose a macrophage sample, and a plasma cell

In [ ]:
cell_types = noDisease_bw_data['Cell type'].unique()
cell_types

array(['band form neutrophil', 'neutrophilic metamyelocyte',
       'neutrophilic myelocyte', 'segmented neutrophil of bone marrow',
       'hematopoietic multipotent progenitor cell', 'precursor B cell',
       'precursor lymphocyte of B lineage', 'plasma cell',
       'mature neutrophil', 'CD38-negative naive B cell',
       'CD14-positive, CD16-negative classical monocyte',
       'CD8-positive, alpha-beta T cell',
       'cytotoxic CD56-dim natural killer cell',
       'inflammatory macrophage', 'erythroblast',
       'CD34-negative, CD41-positive, CD42-positive megakaryocyte cell',
       'macrophage', 'endothelial cell of umbilical vein (proliferating)',
       'endothelial cell of umbilical vein (resting)',
       'alternatively activated macrophage',
       'conventional dendritic cell',
       'CD3-negative, CD4-positive, CD8-positive, double positive thymocyte',
       'CD3-positive, CD4-positive, CD8-positive, double positive thymocyte',
       'CD4-positive, alpha-beta thym

Get example of macrophage and plasma data, and extract its url.

In [ ]:
cell_type_1 = 'macrophage'
cell_type_2 = 'plasma cell'

assert cell_type_1 in cell_types, "{} not a valid cell type".format(cell_type_1)
assert cell_type_2 in cell_types, "{} not a valid cell type".format(cell_type_2)

cell1_data = noDisease_bw_data[noDisease_bw_data['Cell type'] == cell_type_1]
cell2_data = noDisease_bw_data[noDisease_bw_data['Cell type'] == cell_type_2]

cell1_call = cell1_data.iloc[0]
cell1_cov = cell1_data.iloc[1]
cell2_call = cell2_data.iloc[0]
cell2_cov = cell2_data.iloc[1]

cell1_call_url = cell1_call['URL']
cell1_cov_url = cell1_cov['URL']
cell2_call_url = cell2_call['URL']
cell2_cov_url = cell2_cov['URL']

cell1_call_filename = cell1_call_url.split("/")[-1]
cell1_cov_filename = cell1_cov_url.split("/")[-1]
cell2_call_filename = cell2_call_url.split("/")[-1]
cell2_cov_filename = cell2_cov_url.split("/")[-1]

Downloading the data at the four URLs! This will take around 10 minutes.

In [ ]:
! wget '$cell1_call_url'
! wget '$cell1_cov_url'
! wget '$cell2_call_url'
! wget '$cell2_cov_url'

--2022-07-02 08:15:27--  http://ftp.ebi.ac.uk/pub/databases/blueprint/data/homo_sapiens/GRCh38/cord_blood/S00BHQ/macrophage/Bisulfite-Seq/CNAG/S00BHQ51.CPG_methylation_calls.bs_call.GRCh38.20160531.bw
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.138|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125016699 (119M) [application/octet-stream]
Saving to: ‘S00BHQ51.CPG_methylation_calls.bs_call.GRCh38.20160531.bw.2’

S00BHQ51.CPG_methyl 100%[===================>] 119.22M   809KB/s    in 2m 37s  

2022-07-02 08:18:04 (779 KB/s) - ‘S00BHQ51.CPG_methylation_calls.bs_call.GRCh38.20160531.bw.2’ saved [125016699/125016699]

--2022-07-02 08:18:04--  http://ftp.ebi.ac.uk/pub/databases/blueprint/data/homo_sapiens/GRCh38/cord_blood/S00BHQ/macrophage/Bisulfite-Seq/CNAG/S00BHQ51.CPG_methylation_calls.bs_cov.GRCh38.20160531.bw
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.138
Connecting to ftp.ebi.ac.uk 

In [6]:
# Annotations & bs_cov / bs_call names
CELL_TYPE_TO_FILE_ID = {
    "Macrophage": ["S00BHQ51"],
    "Plasma_cell": ["G202"]
}

# Reverse mapping of file id -> cell type
# e.g.  'S01BHIA1': 'Monocyte'
FILE_ID_TO_CELL_TYPE = {sample:cell_type for cell_type, sample_list in CELL_TYPE_TO_FILE_ID.items() for sample in sample_list}

# If the blueprint dict changes, we need to replace our cache files
# This is a tiny checksum of the dictionary state, which we incorporate into
# our cache filenames below.
CELL_TYPE_DICT_SIG = str(hex(abs(hash(json.dumps(CELL_TYPE_TO_FILE_ID, sort_keys=True))))[2:10])
print(f"Dictionary signature for cache files: {CELL_TYPE_DICT_SIG}\n")


BLUEPRINT_FILEKEYS = list(itertools.chain.from_iterable(CELL_TYPE_TO_FILE_ID.values()))

# Validity testing
# assert all(len(vals) > 1 for vals in CELL_TYPE_TO_FILE_ID.values()), "We need more than one example per cell type."
assert len(BLUEPRINT_FILEKEYS) == len(set(BLUEPRINT_FILEKEYS)), "One filename is duplicated in the cell types"

print(f"Number of Blueprint cell types: {len(CELL_TYPE_TO_FILE_ID.keys())}")
print(f"Number of Blueprint raw files: {len(BLUEPRINT_FILEKEYS)}")

Dictionary signature for cache files: 3b0ee3fc

Number of Blueprint cell types: 2
Number of Blueprint raw files: 2


# Create shared **cov** map

Load coverage (if >minimum below) across all files, then determine the set() shared across **all** samples.

IF a locus (e.g. "chr1:123") is missing from **one** single sample, it will be **excluded** from our entire analysis.

IF a locus has \<20 reads in **one** single sample, it will be **excluded** from our entire analysis.

In [ ]:
CHROMOSOMES = ["chr" + str(i) for i in range(1, 23)] + ["chrX"]
IGNORE_CACHE = False

BLUEPRINT_CPG_COV_MINIMUM = 35

RUN_SIGNATURE = f"{BLUEPRINT_CPG_COV_MINIMUM}_{CELL_TYPE_DICT_SIG}"

assert type(BLUEPRINT_CPG_COV_MINIMUM) is int
assert BLUEPRINT_CPG_COV_MINIMUM > 0

print(f"Minimum Blueprint coverage limit: {BLUEPRINT_CPG_COV_MINIMUM}")
print(f" (CpGs with fewer than {BLUEPRINT_CPG_COV_MINIMUM} reads in *any* sample will be ignored.)\n")

# Our output / save file
INTERSECTED_COVERAGE_BED = f"intersected_bs_cov_min_{RUN_SIGNATURE}.bed"
CLEAN_INTERSECTED_COVERAGE_BED = f"intersected_bs_cov_min_{RUN_SIGNATURE}_clean.bed"

print(f"Coverage BED: {INTERSECTED_COVERAGE_BED}")

if os.path.exists(INTERSECTED_COVERAGE_BED) and not IGNORE_CACHE:
    print("\tPost-processed cov .bed already exists. (Skipping raw Blueprint bs_cov parsing.)")
else:
    print("\t.bed does not exist yet -- parsing bs_cov .bw files.")
    INTERSECTED_BS_COV_POSITIONS = {}
    MAX_CHROMOSOME_SIZE = {}

    for file_key in tqdm(BLUEPRINT_FILEKEYS):
        with pyBigWig.open(file_key + ".CPG_methylation_calls.bs_cov.GRCh38.20160531.bw") as bw_object:
            bw_header = bw_object.header()
            if bw_header['nBasesCovered'] < 1e7 or bw_header['sumData'] < 1e8:
                print('\t\t*** WARNING: Input .bw has few reads or low coverage. This may cause unexpected results, consider removing this file.')
                print(f"\t\tnBasesCovered: {bw_header['nBasesCovered']}, sumData: {bw_header['sumData']}")
            for chrom in CHROMOSOMES:
                current_loop_values = set([start for start, _, cov in bw_object.intervals(chrom) if cov >= BLUEPRINT_CPG_COV_MINIMUM])
                MAX_CHROMOSOME_SIZE[chrom] = np.max(list(current_loop_values))
                existing_values = INTERSECTED_BS_COV_POSITIONS.get(chrom, current_loop_values)
                INTERSECTED_BS_COV_POSITIONS[chrom] = existing_values.intersection(current_loop_values)
            # print(track)

    # Save this hard work as a .bed for later recovery if needed
    with open(INTERSECTED_COVERAGE_BED, 'w') as outfile:
        # A .bed is just a .tsv with ['chrom', 'chromStart', 'chromEnd']
        bed_writer = csv.writer(outfile, delimiter='\t')
        for chr in CHROMOSOMES:
            for entry in INTERSECTED_BS_COV_POSITIONS[chr]:
                bed_writer.writerow([chr, entry, entry+1])

    print(f"\nWrote data to: {INTERSECTED_COVERAGE_BED}") # Unsorted

! cat {INTERSECTED_COVERAGE_BED} | tr -d '\r' > {CLEAN_INTERSECTED_COVERAGE_BED}

Minimum Blueprint coverage limit: 20
 (CpGs with fewer than 20 reads in *any* sample will be ignored.)

Coverage BED: intersected_bs_cov_min_20_3a29e61a.bed
	.bed does not exist yet -- parsing bs_cov .bw files.


  0%|          | 0/2 [00:00<?, ?it/s]


Wrote data to: intersected_bs_cov_min_20_3a29e61a.bed


# Removing ENCODE Regions (Doesn't Work)

Encode defines a standard list of "bad" regions -- very low complexity / information content, etc. that show up in some studies but are not informative. We remove them and do some other standard data cleaning.

In [ ]:
# Load the .bed from above
raw_bs_cov_bed = pybedtools.BedTool(CLEAN_INTERSECTED_COVERAGE_BED)
print(f"Number of entries in bs_cov (raw): {len(raw_bs_cov_bed):,}")

if not os.path.exists("ENCFF356LFX.bed.gz"):
    print("Downloading ENCODE DAC Exclusion List")
    urllib.request.urlretrieve("https://www.encodeproject.org/files/ENCFF356LFX/@@download/ENCFF356LFX.bed.gz", "ENCFF356LFX.bed.gz")

! gunzip ENCFF356LFX.bed.gz
excluded_regions = pybedtools.BedTool("ENCFF356LFX.bed")
print(f"Number of entries in excluded_regions: {len(excluded_regions):,}")
assert len(excluded_regions) > 900 # 910 as of 1/2022

# .saveas forces this to render, otherwise may be a generator
# TODO: Fix path to be more specific
cleaned_sorted_bs_cov = raw_bs_cov_bed.subtract(excluded_regions).sort().saveas('bs_cov_cleaned_sorted.bed')
print(f"Number of remaining bs_cov entries: {len(cleaned_sorted_bs_cov):,}")
# assert len(cleaned_sorted_bs_cov) > 90000 # We expect about 90k entries


# Convert `bs_cov_cleaned_sorted` to a dict too:
#  key: chr
#  val: [sorted list of bs_cov positions]
# NOTE: This works because dicts are insertion ordered as of Python>3.7
BS_COV_POSITIONS = {}
for bed_entry in cleaned_sorted_bs_cov:
    BS_COV_POSITIONS.setdefault(bed_entry.chrom, []).append(bed_entry.start)

Number of entries in bs_cov (raw): 14,535,030
gzip: ENCFF356LFX.bed already exists; do you wish to overwrite (y or n)? y
Number of entries in excluded_regions: 910
Number of remaining bs_cov entries: 14,487,018


# Load bs_call data

In [ ]:
FILE_ID_TO_CPG_CALLS = { }

# cache_file = "bs_call_min_" + str(BLUEPRINT_CPG_COV_MINIMUM) + ".json"
print("Parsing bs_call files.")
for file_key in tqdm(BLUEPRINT_FILEKEYS):
    print(f"{file_key}")
    FILE_ID_TO_CPG_CALLS[file_key] = {}
    with pyBigWig.open(file_key + ".CPG_methylation_calls.bs_call.GRCh38.20160531.bw") as bw_object:
        for chrom in CHROMOSOMES:
            # This is more nuanced than the bs_cov data, since we only want to look at the 
            # CpGs that were covered across all samples. (The intervals now in BS_COV_POSITIONS).

            # Each .bw interval is a nested tuple of: ((start, end, value))
            # We extract all the values that overlap our bs_cov set.
            # FILE_ID_TO_CPG_CALLS[file_key][chrom] = [bw_object.intervals(chrom, pos, pos+1)[0][2] for pos in INTERSECTED_BS_COV_POSITIONS[chrom]]
            temp = []
            for pos in BS_COV_POSITIONS[chrom]:
                temp.append(bw_object.intervals(chrom, pos, pos+1)[0][2])
                FILE_ID_TO_CPG_CALLS[file_key][chrom] = temp

SCALED_BS_COV_POSITIONS = {}
for chrom in CHROMOSOMES:
    SCALED_BS_COV_POSITIONS[chrom] = np.array(BS_COV_POSITIONS[chrom]) / MAX_CHROMOSOME_SIZE[chrom]

Parsing bs_call files.


  0%|          | 0/2 [00:00<?, ?it/s]

S00BHQ51
G202


# Checkpoint: Download the Relevant Files

In [ ]:
pickle.dump(SCALED_BS_COV_POSITIONS, open("scaled_bs_cov_positions.p", "wb"))
pickle.dump(FILE_ID_TO_CPG_CALLS, open("file_id_to_cpg_calls.p", "wb"))
files.download("scaled_bs_cov_positions.p")
files.download("file_id_to_cpg_calls.p")

Upload the files into your Google Drive. I put them in the main directory, although you could put them somewhere else. Just change the path below to wherever the file is!

In [1]:
SCALED_BS_COV_POSITIONS = pickle.load(open("/content/drive/My Drive/scaled_bs_cov_positions.p", "rb"))
FILE_ID_TO_CPG_CALLS = pickle.load(open("/content/drive/My Drive/file_id_to_cpg_calls.p", "rb"))

NameError: ignored

# Visualize the Methylation Data

In [64]:
line_df = pd.DataFrame(columns = ["cell type", "location_percent", "methylation"])

CHROMOSOMES = ["chr2"]

for file_key in BLUEPRINT_FILEKEYS:
    for chrom in CHROMOSOMES:
        lengt = len(SCALED_BS_COV_POSITIONS[chrom])
        temp = [[(SCALED_BS_COV_POSITIONS[chrom])[i], (FILE_ID_TO_CPG_CALLS[file_key][chrom])[i]] for i in range(lengt)]
        temp_df = pd.DataFrame(temp, columns = ["location_percent", "methylation"])
        temp_df_avg = temp_df.groupby(np.arange(len(temp_df))//5000).mean()
        temp_df_avg["cell type"] = FILE_ID_TO_CELL_TYPE[file_key]
        line_df = pd.concat([line_df, temp_df_avg], axis=0)

In [66]:
alt.data_transformers.enable('default', max_rows=None)

alt.Chart(line_df).mark_line().encode(
    x='location_percent',
    y='methylation',
    color='cell type'
).interactive()

alt.Chart(...)